In [1]:
# Pandas
import pandas as pd

# re
import re 

# 한국어 형태소 분석기
from konlpy.tag import Kkma, Komoran, Okt, Mecab

okt = Okt()
kkm = Kkma()
kom = Komoran()

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

### DataFrame 행 넓이 최대 
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-d2ea974c1b2d>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## 사용한 함수, 모델 정보

- 전처리, 형태소 분석 : okt.morphs
- stopwords : ./stopwords-ko.txt (카톡방에 공유된 파일)
- 벡터화: TfidfVectorizer()
- 분류모델: NMF(n_components=25)
- 유사도분석: cosine_similarity, linear_kernel

## 데이터 불러오기

In [78]:
## 크롤링한 데이터 가져오기 
df= pd.read_excel('blind_10s.xlsx')

In [79]:
df

,Unnamed: 0,contents,time,keyword
0,0,라고 하기엔 3개 뿐이지만...😅 배우기 시작한지는 얼마 안되서 아직은 기초단계...,작성시간\n2020.08.27.,디지털 드로잉
1,1,아이패드 vs 타블렛 뭐가 나을까?? 클래스 101 인가 그거 들을만 할까 들어본 형 누나들??,작성시간\n2020.07.08.,디지털 드로잉
2,2,손으로 그리는 강의 추천하는 거 있나요?,작성시간\n2020.08.29.,디지털 드로잉
3,3,그림 그리는 모임 경험해보고 싶어요 혼자 그리고 보니 발전이 좀 없는거 같아서 스터디 식으로 크로키나 인체 연습 과제처럼 하는 그런 모임 없을까요~?,작성시간\n04.12,디지털 드로잉
4,4,요즘 반복되는 업무에 치이다보니 취미(좀 정적인것) 도전해보고 싶은데요 혹시 일산&합정&DMC 쪽에 취미로 해볼 수 있는 디지털드로잉 수업이나 강좌 있나요? 한번도 안해봐서 아직 장비는 없고 검색해봐두 전문학원이나 입시학원밖에 없어서요ㅠㅠ 이번에 D뮤지엄갔다가 디지털 드로잉에 꽂혀서 꼭 배워보고싶네요!! 취미로 하시는 분들 계시면 댓글 부탁드려요!^^,작성시간\n2019.06.12.,디지털 드로잉
...,...,...,...,...
5493,5493,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
5494,5494,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육
5495,5495,"생각의 흐름대로 편하게 적기 위해 간단체로 쓰니 양해바랍니다. ---------------------------------- 우연히 아래와 같이 너무 걱정되어 잠이 안온다는 하소연 글을 보게 되었다. 본문 이미지 나도 저런 신혼부부시절이 있었는데 벌써 시간이 많이 흘렀다. 세월 참 빠르다. 대학 시험 볼때, 결혼할 때, 아이 낳을 때가 어제 같은데 말이다. 위 글을 읽으면서 그 시간을 지나온 평범한 한 아빠로서 몇 생각이 들어 적어본다. 1. 아직 젊은데 너무 걱정할 것 없다. 언론에서 집값 상승한다고 너무 떠들기는 한",작성시간\n2020.07.02.,아동 교육
5496,5496,"#오늘의뉴스 2020년 9월7일 1. 당정청, 7조원대 4차 추경 합의 → 전 국민 50%가량 지원 받을 듯. 특수고용직 등 고용 취약계층, 매출 감소 소상공인, 저소득층 긴급생계비, 통신비 지원 등 2500만명선.(국민) 2. 전체 범죄 감소하는데 사기 범죄는 늘어 → 2018년 전체 범죄는 4.8% 감소... 민생형 범죄인 온라인 거래, 부동산 사기등 사기범죄는 오히려 15.2% 늘어. <2019 범죄백서> 발표. 솜방이이 처벌도 원인. (세계) 3. ‘테스 형!’ → 나훈아 신곡 제목. 여기서 ‘테스 형’은 소크",작성시간\n2020.09.07.,아동 교육


In [3]:
## class101 강의 카테고리 목록 (=검색 키워드)

category_list = ['디지털 드로잉', '드로잉', '공예', '요리', '음료', '베이킹', '디저트', '음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']
['음악', '운동', '라이프', '사진', '영상', '디자인', '개발', '데이터', '직무교육', '글쓰기', '언어', '외국어', '금융', '재테크', '창업', '부업', '성공 마인드', '아동 교육']


['음악',
 '운동',
 '라이프',
 '사진',
 '영상',
 '디자인',
 '개발',
 '데이터',
 '직무교육',
 '글쓰기',
 '언어',
 '외국어',
 '금융',
 '재테크',
 '창업',
 '부업',
 '성공 마인드',
 '아동 교육']

In [4]:
## stopwords 파일 불러오기
## open('파일경로.txt', 'rt', encoding='UTF8')
file_path = "./stopwords-ko.txt"

with open(file_path, 'rt', encoding='UTF8') as f:
    lines = f.read().splitlines()

stopwords = lines

In [5]:
## 데이터 전처리 함수 
def text_process(text):
    t1 =re.sub('[^A-Za-z0-9가-힣]', '', text)
    output = re.sub(r'\d+', '', t1)
    output = okt.morphs(output)
    return [x for x in output if x not in stopwords]

In [6]:
## 전처리 테스트 
text_process('123###안녕하세요저는학생기말프로젝트')

['안녕하세요', '는', '학생', '기', '말', '프로젝트']

## 벡터화

In [9]:
## 사람들의 게시글에 해당하는 'contents' 부분만 사용 => list로 저장
docs = list(df['contents'])

In [10]:
## 데이터 확인
docs[25]

'잔다 질문 받는다'

In [32]:
## 미리 만들어준 전처리 함수(text_process)를 사용하여 전체 contents 전처리

docs_list = [' '.join(text_process(s)) for s in docs]

In [33]:
## 전처리한 데이터 확인
docs_list[:10] 

['라고 하기엔 개 뿐 배우기 시작 한지 는 안되서 아직 은기 초단 계',
 '아이패드 vs 타블렛 뭐 나을까 클래스 인가 그거 만 할까 들어 본형 누나',
 '손 그리는 강의 추천 하는거있나요',
 '그림 그리는 모임 경험 해보고싶어요 보니 발전 없는거 같아서 스터디 식 크로키 인체 연습 과제 처럼 하는 그런 모임 없을까요',
 '요즘 반복 되는 업무 치 이다 보니 취미 정적 인 도전 해보고싶은데 요 일산 합정 DMC 쪽 취미 해볼수있는 디지털 드로잉 수업 이나 강 좌있나요 한번 도안 해봐서 아직 장비 는 없고 검색 해봐두 전 문학 원 이나 입시 학원 밖에 없어서요 D 뮤 지엄 갔다가 디지털 드로잉 꽂혀서 꼭 배워 보고싶네요 취미 하시는 분 계시 면 댓글 부탁드려요',
 '학창시절 보면 남자 애 그림 못 그리잖아 글씨 도 더럽고 그게 난 데 그림 그리는걸 너무하고싶어 근데 도화지 에그 리자신 은 없어 디지털 드로잉 유튜브 강의 많던데 보고 면 많이 늘까',
 '갤럭시탭 최신 형 있어서 다른건 다 필요없고 디지털 드로잉 배워 보고 싶어서 아이패드 살려 고하 거든 최신 형 필요없고아이 팬슬 지 원하는거 사면 좋을꺼 같은데 걸사 면 좋을까 아이패드 세대 개제 일무 난할까',
 '손 그림 만 그리다가 아이패드 샀는데 어플 추천 해줄수있 어프 로크리 에이트 클립 스튜디오 많이 쓰는것 같은데 디지털 드로잉 은 처음 이고 포토샵 정도 다룰줄 알 유튜브 랑 인터넷 찾아보는데 엄청 방대하네',
 '아이패드 세대 vs 아이패드 프로 세 대중 어떤게나 까 디지털 드로잉 배워 보려고하는데 초보 라 프로 는 에바 인가 프로 는 가격 세대 는 가격 은 괜찮은데 기능 까는 리뷰 많아서 뭐 가나 을지 고민 즁',
 '보통 아이패드 많이 쓰더라 구나 는 갤 탭 뿐 이고 갤 탭 한번 해보고싶은데 어디서 시작 해야 할지 모르겠어 보통강 같은걸 시작 하나미 대 준비 했었어서 그림 은 잘 그리는 편 인데 디지털 드로잉 은 잘 몰라 조언 부탁 행']

## bag of words로 변환

In [193]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [194]:
tfidf_vectorizer.fit(docs_list)

TfidfVectorizer()

In [195]:
print(tfidf_vectorizer.vocabulary_)

{'라고': 8637, '하기엔': 27181, '배우기': 11907, '시작': 15887, '한지': 27666, '안되서': 17222, '아직': 17084, '은기': 20238, '초단': 25103, '아이패드': 17073, 'vs': 779, '타블렛': 25902, '나을까': 4602, '클래스': 25799, '인가': 20937, '그거': 3288, '할까': 27713, '들어': 7869, '본형': 12837, '누나': 5586, '그리는': 3457, '강의': 1602, '추천': 25213, '하는거있나요': 27224, '그림': 3484, '모임': 10656, '경험': 2363, '해보고싶어요': 28010, '보니': 12526, '발전': 11760, '없는거': 18381, '같아서': 1719, '스터디': 15685, '크로키': 25750, '인체': 21036, '연습': 18802, '과제': 2803, '처럼': 24954, '하는': 27209, '그런': 3368, '없을까요': 18508, '요즘': 19637, '반복': 11525, '되는': 7267, '업무': 18318, '이다': 20480, '취미': 25310, '정적': 22860, '도전': 6954, '해보고싶은데': 28013, '일산': 21134, '합정': 27875, 'dmc': 123, '해볼수있는': 28112, '디지털': 8202, '드로잉': 7640, '수업': 15403, '이나': 20459, '좌있나요': 23393, '한번': 27623, '도안': 6906, '해봐서': 28121, '장비': 22262, '없고': 18342, '검색': 2205, '해봐두': 28119, '문학': 10957, '입시': 21374, '학원': 27552, '밖에': 11495, '없어서요': 18456, '지엄': 24030, '갔다가': 1558, '꽂혀서': 4104, '배워': 11976, '보고싶네요'

In [196]:
# docs를 변형
x_data = tfidf_vectorizer.transform(docs_list)
print(x_data)

  (0, 27666)	0.3013830891822663
  (0, 27181)	0.37779545884191534
  (0, 25103)	0.4195497122680579
  (0, 20238)	0.4195497122680579
  (0, 17222)	0.36021876392005253
  (0, 17084)	0.24434461507858368
  (0, 15887)	0.20081255850310367
  (0, 11907)	0.36021876392005253
  (0, 8637)	0.23420036800636262
  (1, 27713)	0.2490739734088622
  (1, 25902)	0.3981198375621018
  (1, 25799)	0.2595962312707078
  (1, 20937)	0.2490739734088622
  (1, 17073)	0.299016452076482
  (1, 12837)	0.3537735803038166
  (1, 7869)	0.237721243477881
  (1, 5586)	0.26469542563085474
  (1, 4602)	0.37103578570574125
  (1, 3288)	0.2788752944787199
  (1, 779)	0.3057025946265203
  (2, 27224)	0.6980630008501788
  (2, 25213)	0.28678695595813447
  (2, 3457)	0.48106515706664627
  (2, 1602)	0.4461363058446972
  (3, 28010)	0.2970765206932979
  :	:
  (5497, 13663)	0.142263436045864
  (5497, 12851)	0.11730120014954319
  (5497, 12817)	0.14003902527974343
  (5497, 12799)	0.13460595849202275
  (5497, 12378)	0.12592738559607558
  (5497, 12209)	0

In [197]:
# x_data 확인
print('Shape of Sparse Matrix: ', x_data.shape)
print('Amount of Non-Zero occurrences: ', x_data.nnz)

Shape of Sparse Matrix:  (5498, 29289)
Amount of Non-Zero occurrences:  114832


In [198]:
# 그런데 여기서 주의해야할 것 하나.
# docs는 현재 document-term matrix 로 구성됨
# NMF를 적용하기 위해서는 term-document matrix가 필요하다.
# 즉, 현재의 Matrix를 transposed matrix(전치행렬)로 바꿔줘야 함.
type(x_data)

scipy.sparse.csr.csr_matrix

In [199]:
# compressed sparse row matrix인 docs를 transpose 한다.
x_data_trans = x_data.transpose()
type(x_data_trans) 

scipy.sparse.csc.csc_matrix

In [200]:
print(x_data_trans)

  (27666, 0)	0.3013830891822663
  (27181, 0)	0.37779545884191534
  (25103, 0)	0.4195497122680579
  (20238, 0)	0.4195497122680579
  (17222, 0)	0.36021876392005253
  (17084, 0)	0.24434461507858368
  (15887, 0)	0.20081255850310367
  (11907, 0)	0.36021876392005253
  (8637, 0)	0.23420036800636262
  (27713, 1)	0.2490739734088622
  (25902, 1)	0.3981198375621018
  (25799, 1)	0.2595962312707078
  (20937, 1)	0.2490739734088622
  (17073, 1)	0.299016452076482
  (12837, 1)	0.3537735803038166
  (7869, 1)	0.237721243477881
  (5586, 1)	0.26469542563085474
  (4602, 1)	0.37103578570574125
  (3288, 1)	0.2788752944787199
  (779, 1)	0.3057025946265203
  (27224, 2)	0.6980630008501788
  (25213, 2)	0.28678695595813447
  (3457, 2)	0.48106515706664627
  (1602, 2)	0.4461363058446972
  (28010, 3)	0.2970765206932979
  :	:
  (13663, 5497)	0.142263436045864
  (12851, 5497)	0.11730120014954319
  (12817, 5497)	0.14003902527974343
  (12799, 5497)	0.13460595849202275
  (12378, 5497)	0.12592738559607558
  (12209, 5497)	0

In [201]:
# x_data_trans 확인
print('Shape of Sparse Matrix: ', x_data_trans.shape)
print('Amount of Non-Zero occurrences: ', x_data_trans.nnz)

Shape of Sparse Matrix:  (29289, 5498)
Amount of Non-Zero occurrences:  114832


## (1) 분류모델: NMF

In [202]:
# NMF 모델 객체 생성
from sklearn.decomposition import NMF
model = NMF(n_components=25)

In [203]:
# NMF 모델 학습
W = model.fit_transform(x_data_trans)

In [204]:
H = model.components_
type(H)

numpy.ndarray

In [205]:
H.shape

(25, 5498)

In [206]:
# index 5 문서의 클러스터링 결과 점수
print(H[:,5])

[0.00392277 0.         0.         0.12992147 0.04403929 0.
 0.         0.00041733 0.         0.0025097  0.         0.
 0.         0.         0.         0.         0.0028691  0.00044652
 0.         0.00720082 0.         0.         0.04539952 0.
 0.        ]


In [207]:
# 전체 문서별 best 점수에 해당하는 클러스터 label
pred_labels = H.argmax(axis=0)
len(pred_labels)

5498

In [208]:
target = list(df["keyword"])
print(target)

['디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '디지털 드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로잉', '드로

In [209]:
# 'target'값(문자열)을 순서대로 '0 ~ 24'와 같은 숫자형으로 변환

num_target = []

for i in target:
    temp = i.replace('디지털 드로잉','0').replace('드로잉','1').replace('공예','2').replace('요리','3').replace('음료','4').replace('베이킹','5').replace('디저트','6').replace('음악','7').replace('운동','8').replace('라이프','9').replace('사진','10').replace('영상','11').replace('디자인','12').replace('개발','13').replace('데이터','14').replace('직무교육','15').replace('글쓰기','16').replace('언어','17').replace('외국어','18').replace('금융','19').replace('재테크','20').replace('창업','21').replace('부업','22').replace('성공 마인드','23').replace('아동 교육','24')
    num_target.append(int(temp))
    
print(num_target)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

## 실제 label과 예측한 cluster 비교

In [210]:
# 세로: 실제 클래스, 가로: 예측 클러스터 

from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(num_target, pred_labels))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,1,0,1,8,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,6,0,16,119,22,3,0,0,10,9,...,0,3,10,0,2,3,0,0,1,3
2,41,2,26,52,5,6,1,4,21,5,...,7,9,18,1,1,4,18,10,0,6
3,39,2,8,14,2,0,1,0,7,3,...,0,2,19,3,0,3,8,120,4,8
4,40,0,106,1,7,4,1,1,6,3,...,1,7,14,5,5,3,9,12,2,2
5,23,2,29,50,1,1,0,1,7,5,...,0,5,64,3,1,2,8,19,1,5
6,29,2,30,4,6,3,0,0,2,4,...,0,2,22,88,3,2,6,11,3,1
7,39,3,50,8,4,3,0,0,7,1,...,3,2,19,0,80,2,7,6,0,7
8,18,2,7,3,2,3,0,1,6,10,...,0,1,22,2,0,1,8,7,1,2
9,62,2,9,6,7,15,2,3,23,2,...,3,7,36,3,3,7,10,19,3,3


In [211]:
cmatrix = confusion_matrix(num_target, pred_labels)

In [212]:
## 일부 비교 
## 각 클러스터의 문서마다 실제 레이블을 확인한 후, 가장 빈도수가 높은 것을 이 크러스터의 레이블로 한다. 
## 즉, 그만큼은 맞춘것이 됨.
cmatrix[:,:10]

array([[  1,   0,   1,   8,   0,   0,   0,   0,   0,   0],
       [  6,   0,  16, 119,  22,   3,   0,   0,  10,   9],
       [ 41,   2,  26,  52,   5,   6,   1,   4,  21,   5],
       [ 39,   2,   8,  14,   2,   0,   1,   0,   7,   3],
       [ 40,   0, 106,   1,   7,   4,   1,   1,   6,   3],
       [ 23,   2,  29,  50,   1,   1,   0,   1,   7,   5],
       [ 29,   2,  30,   4,   6,   3,   0,   0,   2,   4],
       [ 39,   3,  50,   8,   4,   3,   0,   0,   7,   1],
       [ 18,   2,   7,   3,   2,   3,   0,   1,   6,  10],
       [ 62,   2,   9,   6,   7,  15,   2,   3,  23,   2],
       [ 51,   1,   5,   3,   2,   4,   1,   0,   6,  99],
       [ 79,   1,  18,   1,   6,  15,   2,   5,  16,   7],
       [ 40,   0,   7,   3,   1,  10,   1,   2,   3,   3],
       [ 34,   2,   8,   6,   2,  12,   0,   9,   6,   2],
       [ 35,   1,   4,   0,   1,  18,   0,   8,   4,   2],
       [ 13,   0,   1,   0,   0, 209,   0,   1,   2,   0],
       [ 33,  17,  15,   7,   3,   3,  44,   3,  14,   2

In [213]:
cmatrix[:,:10].argmax(axis=0)

array([11, 16,  4,  1,  1, 15, 16, 18,  9, 10], dtype=int64)

## 전체 결과

In [214]:
cmatrix.argmax(axis=0)

array([11, 16,  4,  1,  1, 15, 16, 18,  9, 10, 14, 22,  8, 20, 13, 12, 16,
        5,  6,  7, 16, 19,  3, 21, 21], dtype=int64)

In [215]:
set(cmatrix.argmax(axis=0))

{1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22}

In [216]:
list(cmatrix.argmax(axis=0))

[11,
 16,
 4,
 1,
 1,
 15,
 16,
 18,
 9,
 10,
 14,
 22,
 8,
 20,
 13,
 12,
 16,
 5,
 6,
 7,
 16,
 19,
 3,
 21,
 21]

In [217]:
best_labels = list(cmatrix.argmax(axis=0))

### index에 따른 키워드 확인을 위해 키워드 테이블을 만들어 준다.

In [218]:
idx = []
for i in range(len(category_list)):
    idx.append(i)
print(idx)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [219]:
keyword_table = pd.DataFrame({"keyword":category_list,"index":idx})
keyword_table

,keyword,index
0,디지털 드로잉,0
1,드로잉,1
2,공예,2
3,요리,3
4,음료,4
5,베이킹,5
6,디저트,6
7,음악,7
8,운동,8
9,라이프,9


### 전체 결과에 대한 keyword 확인

In [220]:
keyword_table["keyword"][[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 18, 20, 21, 22]]

1     드로잉 
2     공예  
3     요리  
4     음료  
5     베이킹 
6     디저트 
7     음악  
8     운동  
10    사진  
11    영상  
13    개발  
14    데이터 
15    직무교육
16    글쓰기 
18    외국어 
20    재테크 
21    창업  
22    부업  
Name: keyword, dtype: object

## argmax label로 매칭했을 때 성능

In [221]:
## Best_labels로매칭했을때 전체 클러스터링 정확도 계산 과정

i = 0
sum = 0
for c in cmatrix.transpose():
    print(c)
    print('label:', best_labels[i])
    print('value:', c[best_labels[i]])
    i += 1
    sum += c[best_labels[i]]

[ 1  6 41 39 40 23 29 39 18 62 51 79 40 34 35 13 33 50 38 47 24 58 20 46
 13]
label: 11
value: 79
[ 0  0  2  2  0  2  2  3  2  2  1  1  0  2  1  0 17  1  2  2  0  2  0  0
  0]
label: 16
value: 17
[  1  16  26   8 106  29  30  50   7   9   5  18   7   8   4   1  15   8
  11  17  12  12   9   2   0]
label: 4
value: 106
[  8 119  52  14   1  50   4   8   3   6   3   1   3   6   0   0   7   5
   3   2   1   0   6   0   0]
label: 1
value: 119
[ 0 22  5  2  7  1  6  4  2  7  2  6  1  2  1  0  3  3  0  2  1  2  4  2
  0]
label: 1
value: 22
[  0   3   6   0   4   1   3   3   3  15   4  15  10  12  18 209   3   5
   3  42   2   9   3   5  14]
label: 15
value: 209
[ 0  0  1  1  1  0  0  0  0  2  1  2  1  0  0  0 44  0  1  1  0  0  0  0
  1]
label: 16
value: 44
[  0   0   4   0   1   1   0   0   1   3   0   5   2   9   8   1   3  71
 138   3   3   1   0   2   0]
label: 18
value: 138
[ 0 10 21  7  6  7  2  7  6 23  6 16  3  6  4  2 14  8 11 10  3  6 10 13
  2]
label: 9
value: 23
[ 0  9  5  3  3  5

IndexError: list index out of range

In [222]:
print(sum)

327


In [223]:
sum/len(target)

0.059476173153874136

## (2) Cosine similarity 로 유사도 구하기

참고사이트 : https://sikaleo.tistory.com/62

In [224]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [225]:
# x_data trans 확인
print('Shape of Sparse Matrix: ', x_data_trans.shape)
print('Amount of Non-Zero occurrences: ', x_data_trans.nnz)

Shape of Sparse Matrix:  (29289, 5498)
Amount of Non-Zero occurrences:  114832


In [227]:
## contents 내용에 대해 tf-idf 수행
tfidf_matrix = tfidf_vectorizer.fit_transform(docs_list)
print(tfidf_matrix.shape)

## 5498개의 컨텐츠를 표현하기 위해 29,289개의 단어가 사용됨
print(type(tfidf_matrix))

(5498, 29289)
<class 'scipy.sparse.csr.csr_matrix'>


In [228]:
# crs_matrix 객체를 저장하고 로드하는 것이 중요
mtx_path = 'name.mtx'

# 새로 학습한 tfidf vector를 crs_matrix형태로 저장
from scipy.io import mmwrite, mmread
mmwrite(mtx_path, tfidf_matrix)

In [229]:
#저장을 했으니 다시 로드해보겠습니다. 
#이 때, csr_matrix 로 저장했으나 로드하면 coo_matrix 입니다. 
#그러니 tocsr() 메서드로 바꿔주시면 됩니다. 
#이터러블 객체를 iter() 써서 이터레이터로 바꾸는 느낌이죠! 

# csr_matrix 객체 로드 
csr_matrix = mmread(mtx_path) 
csr_matrix = csr_matrix.tocsr()

In [230]:
features = tfidf_vectorizer.get_feature_names()
len(features)

29289

### 새로운 데이터 입력 받아 기존 문서와 유사도 구하기

In [248]:
# input에 데이터 입력
new_data = input()

아이패드 드로잉 추천해주세요


In [249]:
# 입력받은 데이터 토큰화
new_data_vector = [t for t in text_process(new_data) if t in features]

In [250]:
print(new_data_list)

['아이패드', '드로잉', '추천', '해주세요']


In [254]:
#이제 코사인 유사도를 구할 수 있도록 벡터화 해보겠습니다. 
#앞에서 썼던 vectorize 를 가져오고 train 된 tfidf 로 transform 해줍니다.
#기존 train 한 tfidf 벡터의 feature 를 가지고 matrix 가 만들어졌습니다. 

new_data_vector = tfidf_vectorizer.transform([new_data])
cosine_similar = linear_kernel(new_data_vector,tfidf_matrix).flatten()
print(new_data_vector.shape)

(1, 29289)


In [255]:
#linear_kernel 을 통해서 코사인 유사도를 구해주고 argsort 로 정렬합니다. 
sim_rank_idx = cosine_similar.argsort()[::-1]
len(sim_rank_idx)

5498

In [260]:
# 새로 입력한 데이터와 유사도가 50%인 컨텐츠에 대해 [내용, 키워드, 유사도]를 순서대로 출력한다. 
for i in sim_rank_idx:
    if cosine_similar[i]>0.5:
        print(df['contents'][i],df['keyword'][i], cosine_similar[i])

#아이패드드로잉 드로잉 1.0
폰은 갤럭시를 씀. 아이패드가 드로잉 더 좋아보이는데 연동이 고민ㅠㅠ 갤럭시 탭 s7플러스 아이패드 프로 11 아이패드 프로 12.9 1) 갤럭시 쓰는데 아이패드 써도 안 불편할까? 2) 아이패드 가지고 다니기에 작은게 편해보이는데 그림 그릴거라니깐 큰게 좋지 않냐는 이야기가 있음 현재 사용 중이시라면.. 추천 좀 해줘요ㅜㅜ 드로잉 0.5223554139545437
